In [135]:
from nltk.corpus.reader.tagged import word_tokenize
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from xgboost import XGBClassifier

In [88]:
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [54]:
proj_dir = "/content/drive/MyDrive/Projects/Disaster_Tweets"
train_df = pd.read_csv(os.path.join(proj_dir,"train.csv"))
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [56]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [57]:
train_df[train_df.keyword.notna()]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


In [58]:
train_df[train_df.location.notna()]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


In [148]:
train_df.target.sum()/train_df.target.count()

0.4296597924602653

In [134]:
for text in train_df.text[:25]:
    print(text,'\n')

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all 

Forest fire near La Ronge Sask. Canada 

All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected 

13,000 people receive #wildfires evacuation orders in California  

Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school  

#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires 

#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas 

I'm on top of the hill and I can see a fire in the woods... 

There's an emergency evacuation happening now in the building across the street 

I'm afraid that the tornado is coming to our area... 

Three people died from the heat wave so far 

Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding 

#rai

In [104]:
def tweet_clean(text):

    #lower case
    text = text.lower()

    #removing web addresses, numbers, symbols, extra spaces
    text = re.sub(r'(http)\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
    text = re.sub(r'(\s+)', ' ', text)

    #removing stopwords
    stop_words = stopwords.words('english')
    listed = text.split(' ')
    text = ' '.join([word for word in listed if word not in stop_words])

    return text|

In [139]:
for row in range(50):
    print(train_df.text[row],'\n',tweet_clean(train_df.text[row]),'\n')

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all 
 deeds reason earthquake may allah forgive us 

Forest fire near La Ronge Sask. Canada 
 forest fire near la ronge sask canada 

All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected 
 residents asked shelter place notified officers evacuation shelter place orders expected 

13,000 people receive #wildfires evacuation orders in California  
  people receive wildfires evacuation orders california  

Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school  
 got sent photo ruby alaska smoke wildfires pours school  

#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires 
 rockyfire update california hwy closed directions due lake county fire cafire wildfires 

#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs area

In [65]:
#initiate count vectorizer
vectorizer = CountVectorizer()

In [143]:
X = vectorizer.fit_transform(train_df.text.apply(tweet_clean))
y = train_df.target
print(X.shape, y.shape)

(7613, 16953) (7613,)


In [145]:
clf = XGBClassifier()
scores = cross_val_score(clf, X, y, cv=5, scoring = 'accuracy')
scores.mean()

0.6297178695827361